In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

from keras import layers, models

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

## Exploratory Data Analysis

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
pd.concat([pd.DataFrame(df_train.nunique()),
           pd.DataFrame(df_train.isnull().sum()),
           pd.DataFrame(df_train.mean())],
          axis=1,
          ignore_index=True)

,0,1,2
Age,88,177,29.699118
Cabin,147,687,NaN
Embarked,3,2,NaN
Fare,248,0,32.204208
Name,891,0,NaN
Parch,7,0,0.381594
PassengerId,891,0,446.000000
Pclass,3,0,2.308642
Sex,2,0,NaN
SibSp,7,0,0.523008


## Feature Engineering

In [5]:
df_train = pd.concat([df_train,
                      pd.get_dummies(df_train['Sex']),
                      pd.get_dummies(df_train['Name'].str.extract('[A-Za-z]*,\s([A-Z][a-z]*)', expand=False).str.strip())],
                     axis=1)

## Train/val split

In [6]:
df_train, df_val = train_test_split(df_train)
print(len(df_train), len(df_val))


668 223


## Naive Baseline

In [7]:
i_fare = layers.Input(shape=(1,), name='Fare')
o = layers.Dense(units=1, activation='sigmoid')(i_fare)

model = models.Model(inputs=[i_fare], outputs=[o])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
model.fit(x=[df_train['Fare']],
          y=[df_train['Survived']])

Epoch 1/1
668/668 [==============================] - 2s 3ms/step - loss: 4.1316 - acc: 0.4012


In [17]:
metrics.roc_auc_score(model.predict(x=[df_val['Fare']]),
                      df_val['Survived'])

ValueError: continuous format is not supported